<a href="https://colab.research.google.com/github/Anpr1211/Dissertation---Time-Series/blob/master/HWESipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import json
import urllib

from statsmodels.tsa.holtwinters import ExponentialSmoothing 

In [0]:
 url = "https://raw.githubusercontent.com/Anpr1211/Dissertation---Time-Series/master/Temp_data.json"
 r = urllib.request.urlopen(url)
 df = json.loads(r.read())

In [20]:
print(df[0][0:5])

[{'date': '2016-01-01', 'temperature': 19.3}, {'date': '2016-01-02', 'temperature': 19.7}, {'date': '2016-01-03', 'temperature': 19.7}, {'date': '2016-01-04', 'temperature': 19.3}, {'date': '2016-01-05', 'temperature': 18.9}]


In [67]:
data = pd.DataFrame(df[0])

data.head()

,date,temperature
0,2016-01-01,19.3
1,2016-01-02,19.7
2,2016-01-03,19.7
3,2016-01-04,19.3
4,2016-01-05,18.9


In [64]:
data.values

array([['2016-01-01', 19.3],
       ['2016-01-02', 19.7],
       ['2016-01-03', 19.7],
       ...,
       ['2019-09-28', 25.2],
       ['2019-09-29', 25.1],
       ['2019-09-30', 27.1]], dtype=object)

In [49]:
temp_2016 = data.loc[0:1095].copy()

print(len(temp_2016))

1096


In [0]:
model = ExponentialSmoothing(temp_2016["temperature"], trend=None, seasonal="add", seasonal_periods=12)

In [0]:
model_fit = model.fit()

In [43]:
model_fit.summary()

Dep. Variable:,endog,No. Observations:,366
Model:,ExponentialSmoothing,SSE,533.675
Optimized:,True,AIC,166.039
Trend:,None,BIC,220.675
Seasonal:,Additive,AICC,167.597
Seasonal Periods:,12,Date:,"Mon, 21 Oct 2019"
Box-Cox:,False,Time:,20:42:00
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.9002307,alpha,True
smoothing_seasonal,0.000000,gamma,True


In [52]:
model_fit.summary()

Dep. Variable:,endog,No. Observations:,1096
Model:,ExponentialSmoothing,SSE,1766.545
Optimized:,True,AIC,551.185
Trend:,None,BIC,621.177
Seasonal:,Additive,AICC,551.689
Seasonal Periods:,12,Date:,"Mon, 21 Oct 2019"
Box-Cox:,False,Time:,20:44:49
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.8811027,alpha,True
smoothing_seasonal,0.000000,gamma,True


In [55]:
model_fit.forecast(7)

1096    17.082550
1097    17.033208
1098    17.072867
1099    16.978453
1100    16.951093
1101    16.812734
1102    16.885380
dtype: float64

In [57]:
data.loc[1096:1110]

,date,temperature
1096,2019-01-02,18.2
1097,2019-01-03,17.6
1098,2019-01-04,16.9
1099,2019-01-05,17.2
1100,2019-01-06,18.6
1101,2019-01-07,18.9
1102,2019-01-08,18.7
1103,2019-01-09,18.1
1104,2019-01-10,17.6
1105,2019-01-11,16.8


In [68]:
# grid search ets hyperparameters for monthly mean temp dataset
from math import sqrt
#from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
from numpy import array

# one-step Holt Winter’s Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	
	scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
	models = list()
	# define config lists
	t_params = ['add', 'mul', None]
	d_params = [True, False]
	s_params = ['add', 'mul', None]
	p_params = seasonal
	b_params = [True, False]
	r_params = [True, False]
	# create config instances
	for t in t_params:
		for d in d_params:
			for s in s_params:
				for p in p_params:
					for b in b_params:
						for r in r_params:
							cfg = [t,d,s,p,b,r]
							models.append(cfg)
	return models

if __name__ == '__main__':
	# load dataset
	#series = read_csv('monthly-mean-temp.csv', header=0, index_col=0)
	#data = series.values
	# trim dataset to 5 years
	data = data.values
	# data split
	n_test = int(365)
	# model configs
	cfg_list = exp_smoothing_configs(seasonal=[0,12])
	# grid search
	scores = grid_search(data[:,0], cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

done


In [69]:
scores

[]